In [51]:
import pandas as pd
import utils 
import plotly.graph_objects as go 
import instrument

In [52]:
pair = "CAD_CHF"
granularity = "H1"
ma_list=[16,64]
i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [53]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ["time", "volume"]
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [54]:
df_ma = df[["time", "mid_o", "mid_h", "mid_l", "mid_c"]].copy()
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [55]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2022-09-23T03:00:00.000000000Z,0.72577,0.72596,0.72554,0.72563,0.725716,0.721941
1,2022-09-23T04:00:00.000000000Z,0.72563,0.72563,0.72490,0.72505,0.725556,0.721925
2,2022-09-23T05:00:00.000000000Z,0.72506,0.72579,0.72503,0.72571,0.725457,0.721938
3,2022-09-23T06:00:00.000000000Z,0.72567,0.72570,0.72452,0.72500,0.725285,0.721964
4,2022-09-23T07:00:00.000000000Z,0.72504,0.72572,0.72438,0.72524,0.725205,0.722018


In [56]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1 
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1 
    return 0

In [57]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE!=0].copy()

In [58]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
25,2022-09-26T04:00:00.000000000Z,0.72249,0.72284,0.72222,0.72244,0.722538,0.722645,-0.000107,0.000034,-1
26,2022-09-26T05:00:00.000000000Z,0.72242,0.72324,0.72214,0.72240,0.722696,0.722680,0.000015,-0.000107,1
28,2022-09-26T07:00:00.000000000Z,0.72409,0.72428,0.72168,0.72214,0.722723,0.722765,-0.000042,0.000027,-1
29,2022-09-26T08:00:00.000000000Z,0.72212,0.72558,0.72210,0.72494,0.722882,0.722824,0.000059,-0.000042,1
37,2022-09-26T16:00:00.000000000Z,0.72206,0.72350,0.71960,0.72350,0.723414,0.723502,-0.000088,0.000021,-1


In [59]:
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [60]:
df_trades["GAIN"].sum()

46.400000000002024

In [61]:
df_plot = df_ma.iloc[0:100].copy()

In [62]:
df_plot.shape

(100, 10)

In [63]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()

AttributeError: module 'plotly.validators.scatter' has no attribute 'LineValidator'